* 1호와 차이점 : 최대주주비율의 pct_change와 최대주주비율 두가지가 들어갔다

* scaler : rank사용   / k=4 / [Q로 민 경우] 라벨 3개의 일치도 0.8648648648648649  // [Y로 민 경우] 라벨 3개의 일치도 1.0
* scaler : minmax사용 / k=4 / [Q로 민 경우] 라벨 3개의 일치도 0.8108108108108109 // [Y로 민 경우] 라벨 3개의 일치도 0.8 

In [39]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [40]:
# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Y"

# 클러스터 수
num_k = 4

# 사용할 스케일러
sca = scaler.rank

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

In [41]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트_전체시장.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트_전체시장.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트_전체시장.csv", index_col=0)

In [42]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [43]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

# 시기별 전체시장
#concat_together = tools.dict_data_match_listed(concat_data.copy(), together_listed, "Y")
# 시기별 코스피
#concat_kospi = tools.dict_data_match_listed(concat_data.copy(), kospi_listed, "Y")
# 시기별 코스닥
#concat_kosdaq = tools.dict_data_match_listed(concat_data.copy(), kosdaq_listed, "Y")

In [44]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(concat_data.copy(),
                                  plus_one="BW금액_전체시장"
                                 ,plus_two="CB금액_전체시장", return_origianl=False)
#concat_kospi1 = tools.dict_data_plus(concat_kospi,
#                                  plus_one="BW금액_전체시장"
#                                 ,plus_two="CB금액_전체시장", return_origianl=False)
#concat_kosdaq1 = tools.dict_data_plus(concat_kosdaq,
#                                  plus_one="BW금액_전체시장"
#                                 ,plus_two="CB금액_전체시장", return_origianl=False)
#
concat_together1["2012-01"].columns

Index(['BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장', '업종변경카운트_전체시장', '유상증자금액_전체시장', '자기주식처분결정금액_전체시장',
       '조회공시요구카운트_전체시장', '최대주주변경을수반하는주식담보제공금액_전체시장', '최대주주변경을수반하는주식양수도금액_전체시장',
       '최대주주변경카운트_전체시장', '타법인주식및출자증권취득결정자기자본대비_전체시장', '특수관계인에대한자금대여금액_전체시장',
       '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본',
       '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [45]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="상호변경카운트_전체시장"
                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['상호변경카운트_전체시장+업종변경카운트_전체시장', 'BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장',
       '불성실공시법인지정카운트_전체시장', '유상증자금액_전체시장', '자기주식처분결정금액_전체시장', '조회공시요구카운트_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장', '최대주주변경을수반하는주식양수도금액_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정자기자본대비_전체시장', '특수관계인에대한자금대여금액_전체시장', '거래량회전율', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [46]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', 'BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장',
       '불성실공시법인지정카운트_전체시장', '유상증자금액_전체시장', '자기주식처분결정금액_전체시장', '조회공시요구카운트_전체시장',
       '최대주주변경카운트_전체시장', '타법인주식및출자증권취득결정자기자본대비_전체시장', '특수관계인에대한자금대여금액_전체시장',
       '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본',
       '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [47]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="자기주식처분결정금액_전체시장"
                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', 'BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장',
       '불성실공시법인지정카운트_전체시장', '유상증자금액_전체시장', '조회공시요구카운트_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익',
       '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [48]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="불성실공시법인지정카운트_전체시장"
                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', 'BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장',
       '유상증자금액_전체시장', '최대주주변경카운트_전체시장', '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [49]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송_전체시장"
                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kospi6 = tools.dict_data_plus(concat_kospi5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kosdaq6 = tools.dict_data_plus(concat_kosdaq5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장', '불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', 'BW금액_전체시장+CB금액_전체시장', '유상증자금액_전체시장',
       '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익',
       '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [50]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="BW금액_전체시장+CB금액_전체시장"
                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장',
       '불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [51]:
concat_together8 = tools.dict_data_plus(concat_together7,
                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together8["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익',
       '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [52]:
concat_together9 = tools.dict_data_plus(concat_together8,
                                  plus_one="자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장"
                                 ,plus_two="최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kospi9 = tools.dict_data_plus(concat_kospi8,
#                                  plus_one="자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq9 = tools.dict_data_plus(concat_kosdaq8,
#                                  plus_one="자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
concat_together9["2012-01"].columns

Index(['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익',
       '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [53]:
concat_together10 = tools.dict_data_plus(concat_together9,
                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장"
                                 ,plus_two="불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장", return_origianl=False)
#concat_kospi10 = tools.dict_data_plus(concat_kospi9,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장"
#                                 ,plus_two="불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq10 = tools.dict_data_plus(concat_kosdaq9,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장"
#                                 ,plus_two="불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장", return_origianl=False)
concat_together10["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '타법인주식및출자증권취득결정자기자본대비_전체시장',
       '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본',
       '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [54]:
concat_together11 = tools.dict_data_plus(concat_together10,
                                  plus_one="타법인주식및출자증권취득결정자기자본대비_전체시장"
                                 ,plus_two="차입금의존도", return_origianl=False)
#concat_kospi11 = tools.dict_data_plus(concat_kospi10,
#                                  plus_one="타법인주식및출자증권취득결정자기자본대비_전체시장"
#                                 ,plus_two="차입금의존도", return_origianl=False)
#concat_kosdaq11 = tools.dict_data_plus(concat_kosdaq10,
#                                  plus_one="타법인주식및출자증권취득결정자기자본대비_전체시장"
#                                 ,plus_two="차입금의존도", return_origianl=False)
concat_together11["2012-01"].columns

Index(['타법인주식및출자증권취득결정자기자본대비_전체시장+차입금의존도',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '거래량회전율', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [55]:
divide_together = tools.dict_data_divide(concat_together11.copy(),divide_what=['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장','당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '판매 및 관리비'], divide_into="총자본", return_origianl=False)
#divide_kospi = tools.dict_data_divide(concat_kospi11,divide_what=['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
#       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장','당기순이익', '매출채권', '매출총이익',
#       '영업이익', '영업현금흐름', '판매 및 관리비'], divide_into="총자본", return_origianl=False)
#divide_kosdaq = tools.dict_data_divide(concat_kosdaq11,divide_what=['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
#       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장','당기순이익', '매출채권', '매출총이익',
#       '영업이익', '영업현금흐름', '판매 및 관리비'], divide_into="총자본", return_origianl=False)

print(len(divide_together["2012-01"].columns))
divide_together["2012-01"].columns

13


Index(['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장/총자본',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장/총자본', '당기순이익/총자본', '매출채권/총자본',
       '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본', '판매 및 관리비/총자본',
       '타법인주식및출자증권취득결정자기자본대비_전체시장+차입금의존도',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '거래량회전율', '최대주주비율', '최대주주비율_변동성'],
      dtype='object')

In [56]:
drop_togehter = tools.dict_data_drop(divide_together.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)

#drop_kospi = tools.dict_data_drop(divide_kospi, dropna=True)
#scaled_kospi = tools.dict_data_scale(drop_kospi, scaler=scaler.rank)

#drop_kosdaq = tools.dict_data_drop(divide_kosdaq, dropna=True)
#scaled_kosdaq = tools.dict_data_scale(drop_kosdaq, scaler=scaler.rank)

In [57]:
# Run_Kmeans
model_save_together = {}
for time, dataframe in scaled_together.items():
    cluster_model = MyKmeans(dataframe)
    cluster_model.set_params()
    #cluster_model.find_optimal_k(max_k=10, max_sample=10, optimize_method="silhouette")
    cluster_model.find_optimal_initp(num_of_cluster= num_k, max_sample=50, optimize_method="silhouette")
    k_mean_dict = cluster_model.run_kmean(num_of_cluster=num_k)
    model_save_together[time] = k_mean_dict

# 코스피
#model_save_kospi = {}
#for time, dataframe in scaled_kospi.items():
#    cluster_model = MyKmeans(dataframe)
#    cluster_model.set_params()
#    #cluster_model.find_optimal_k(max_k=10, max_sample=10, optimize_method="silhouette")
#    cluster_model.find_optimal_initp(4, max_sample=50, optimize_method="silhouette")
#    k_mean_dict = cluster_model.run_kmean(4)
#    model_save_kospi[time] = k_mean_dict
#
## 코스닥
#model_save_kosdaq = {}
#for time, dataframe in scaled_kosdaq.items():
#    cluster_model = MyKmeans(dataframe)
#    cluster_model.set_params()
#    #cluster_model.find_optimal_k(max_k=10, max_sample=10, optimize_method="silhouette")
#    cluster_model.find_optimal_initp(4, max_sample=50, optimize_method="silhouette")
#    k_mean_dict = cluster_model.run_kmean(4)
#    model_save_kosdaq[time] = k_mean_dict

100%|██████████| 50/50 [00:03<00:00, 13.27it/s]


In [58]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [59]:
def eval_cluster_size(model_save:dict):
    return_df = pd.DataFrame()
    for t, model_dict in model_save.items():
        a = pd.Series(model_save[t]["model"].labels_).value_counts()
        a = pd.DataFrame(a)
        a.columns = [t]
        return_df = pd.concat([return_df, a], axis=1)
    return_df.loc["기업수",:] = return_df.sum(axis=0)
    return return_df
    
def eval_cluster_result(model_save:dict, raw_data:dict, label_data:dict, freq=freq, return_best_cluster=True):
    freq = freq + "S"
    date_range = pd.date_range("2012-01-01","2021-01-01", freq=freq)
    return_df = pd.DataFrame()
    for date in date_range:
        date = str(date)[:7]
        eval = Optimizer.eval(model_save[date]["model"], raw_data[date], label_data[date])
        df = pd.DataFrame(eval.values(), eval.keys(), columns=[date])
        return_df = pd.concat([return_df,df], axis=1)
    if return_best_cluster:
        return_df.loc["포착군집", :] = np.argmax(return_df.values, axis=0)
    return return_df

def eval_cluster_distance(prob_df, raw_data):
    ser = prob_df.loc["포착군집"].copy()
    tmp_dict = {}
    return_dict = {}
    for t,danger_cluster in ser.iteritems(): #포착군집을 반복한다
        t = str(t)
        num = int(danger_cluster)
        distance = cluster_model.distance_decomposition(model_save_together[t]["model"])[num] #최대확률 클러스터와 타 클러스터의 거리
        tmp_dict[t] = distance # 시기:거리

    for t, distance_matrix in tmp_dict.items():
        distance_sum = np.zeros(raw_data[t].columns.shape, dtype='float64')
        for i in range(len(distance_matrix)):
            slice = distance_matrix[i]
            if len(slice) ==0:
                continue
            distance_sum += slice
        return_dict[t] = distance_sum
        col = raw_data[t].columns
    
    return_df = pd.DataFrame(return_dict).T
    return_df.columns = col
    return_df_sq = return_df.copy()**2
    sum_square = return_df_sq.sum(axis=1)
    
    
    return return_df

In [60]:
# 이전의 Model로 이후의 데이터 클러스터링을 평가
#Optimizer.eval(model_save_together["2021-01"]["model"], scaled_together["2022-01"], label_data_pp1["2022-01"])

# 예측
#pd.DataFrame(model_save_together["2021-01"]["model"].predict(scaled_together["2022-07"])).value_counts()

### 전체 기업 평가

In [61]:
pd.set_option('display.max_columns', 130)

In [62]:
eval_cluster_size(model_save_together)#.max(axis=1)

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01,2022-01
0,450.0,404.0,490.0,408.0,489.0,413.0,437.0,516.0,540.0,520.0,456.0
2,444.0,467.0,455.0,463.0,443.0,591.0,576.0,518.0,497.0,548.0,579.0
3,423.0,338.0,428.0,493.0,409.0,451.0,359.0,451.0,509.0,587.0,558.0
1,353.0,459.0,323.0,392.0,470.0,441.0,589.0,525.0,526.0,465.0,538.0
기업수,1670.0,1668.0,1696.0,1756.0,1811.0,1896.0,1961.0,2010.0,2072.0,2120.0,2131.0


In [63]:
eval_one = eval_cluster_result(model_save_together, raw_data = scaled_together, label_data=label_data_pp1)
eval_one

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.0875,0.869565,0.015873,0.903226,0.067797,0.051282,0.572368,0.029762,0.053030,0.077670
1,0.0875,0.021739,0.285714,0.048387,0.847458,0.051282,0.032895,0.047619,0.022727,0.106796
2,0.7500,0.043478,0.650794,0.016129,0.016949,0.089744,0.046053,0.886905,0.159091,0.737864
3,0.0750,0.065217,0.047619,0.032258,0.067797,0.807692,0.348684,0.035714,0.765152,0.077670
포착군집,2.0000,0.000000,2.000000,0.000000,1.000000,3.000000,0.000000,2.000000,3.000000,2.000000


In [64]:
eval_two = eval_cluster_result(model_save_together, raw_data = scaled_together, label_data=label_data_pp2)
eval_two

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.00,0.722222,0.000000,0.952381,0.153846,0.065217,0.500000,0.010753,0.023256,0.050000
1,0.12,0.000000,0.241379,0.000000,0.769231,0.000000,0.014286,0.064516,0.069767,0.083333
2,0.88,0.166667,0.758621,0.047619,0.000000,0.130435,0.114286,0.870968,0.116279,0.750000
3,0.00,0.111111,0.000000,0.000000,0.076923,0.804348,0.371429,0.053763,0.790698,0.116667
포착군집,2.00,0.000000,2.000000,0.000000,1.000000,3.000000,0.000000,2.000000,3.000000,2.000000


In [65]:
# 레이블 두개를 전부 사용한 경우의 평균 확률 -> 아예 label_data를 더해서 확률을 구하는 방법도 있음
eval_avg = (eval_one.iloc[:-1,:] + eval_two.iloc[:-1,:]) / 2
eval_avg.loc["포착군집", :] = np.argmax(eval_avg.values, axis=0)

eval_avg

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.04375,0.795894,0.007937,0.927803,0.110821,0.058250,0.536184,0.020257,0.038143,0.063835
1,0.10375,0.010870,0.263547,0.024194,0.808344,0.025641,0.023590,0.056068,0.046247,0.095065
2,0.81500,0.105072,0.704707,0.031874,0.008475,0.110089,0.080169,0.878936,0.137685,0.743932
3,0.03750,0.088164,0.023810,0.016129,0.072360,0.806020,0.360056,0.044739,0.777925,0.097168
포착군집,2.00000,0.000000,2.000000,0.000000,1.000000,3.000000,0.000000,2.000000,3.000000,2.000000


In [66]:
# 횡령인 경우
eval_three = eval_cluster_result(model_save_together, raw_data = scaled_together, label_data=label_data_pp3)
eval_three

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.125,0.642857,0.076923,0.565217,0.304348,0.09375,0.568627,0.076923,0.069767,0.125000
1,0.125,0.071429,0.153846,0.086957,0.521739,0.03125,0.078431,0.019231,0.116279,0.041667
2,0.625,0.285714,0.692308,0.173913,0.130435,0.28125,0.058824,0.865385,0.093023,0.708333
3,0.125,0.000000,0.076923,0.173913,0.043478,0.59375,0.294118,0.038462,0.720930,0.125000
포착군집,2.000,0.000000,2.000000,0.000000,1.000000,3.00000,0.000000,2.000000,3.000000,2.000000


In [67]:
# 일치 확률 평가
idx = eval_one.columns
data = np.c_[eval_one.loc["포착군집",:].values, eval_two.loc["포착군집",:].values, eval_three.loc["포착군집",:].values]

# 어디서 일차하지 않는 지 한눈에 확인 가능
df = pd.DataFrame(data=data, index=idx, columns=["라벨1","라벨2","라벨3"])


con1 = (df["라벨1"] == df["라벨2"])
con2 = (df["라벨3"] == df["라벨1"])

# 성능지표
np.mean(con1 == con2)

1.0

In [68]:
# 고위험 클러스터의 평균 확률 구하기
pro1 = eval_one.iloc[0:-1,:].max(axis=0).mean()
pro2 = eval_two.iloc[0:-1,:].max(axis=0).mean()
pro3 = eval_three.iloc[0:-1,:].max(axis=0).mean()

np.c_[pro1,pro2,pro3].mean()

0.736454608343898

In [69]:
model_save_together2 = {}
return_masked_data = {}

for t, num in eval_avg.iloc[-1,:].iteritems():
    num = int(num)
    mask = model_save_together[t]["model"].labels_ == num
    masked_data = scaled_together[t][mask]
    
    cluster_model = MyKmeans(masked_data)
    cluster_model.set_params()
    #cluster_model.find_optimal_k(max_k=10, max_sample=10, optimize_method="silhouette")
    cluster_model.find_optimal_initp(num_of_cluster= num_k, max_sample=50, optimize_method="silhouette")
    k_mean_dict = cluster_model.run_kmean(num_of_cluster=num_k)
    return_masked_data[t] = masked_data
    model_save_together2[t] = k_mean_dict

100%|██████████| 50/50 [00:00<00:00, 84.30it/s]


In [70]:
eval_cluster_size(model_save_together2)#.max(axis=1)

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
1,129.0,85.0,104.0,120.0,144.0,102.0,104.0,125.0,140.0,155.0
0,107.0,77.0,71.0,81.0,115.0,112.0,109.0,134.0,144.0,134.0
3,106.0,126.0,157.0,75.0,103.0,96.0,128.0,131.0,88.0,120.0
2,102.0,116.0,123.0,132.0,108.0,141.0,96.0,128.0,137.0,139.0
기업수,444.0,404.0,455.0,408.0,470.0,451.0,437.0,518.0,509.0,548.0


In [71]:
prob_df1 = eval_cluster_result(model_save_together2, raw_data = scaled_together, label_data=label_data_pp1)
prob_df1

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.1500,0.130435,0.031746,0.306452,0.355932,0.243590,0.092105,0.244048,0.318182,0.174757
1,0.2875,0.565217,0.555556,0.161290,0.084746,0.243590,0.572368,0.345238,0.386364,0.368932
2,0.3750,0.152174,0.365079,0.112903,0.118644,0.358974,0.243421,0.273810,0.174242,0.320388
3,0.1875,0.152174,0.047619,0.419355,0.440678,0.153846,0.092105,0.136905,0.121212,0.135922
포착군집,2.0000,1.000000,1.000000,3.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000


In [72]:
prob_df2 = eval_cluster_result(model_save_together2, raw_data = scaled_together, label_data=label_data_pp2)
prob_df2

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.08,0.277778,0.034483,0.095238,0.358974,0.152174,0.057143,0.193548,0.186047,0.200000
1,0.24,0.444444,0.482759,0.095238,0.102564,0.239130,0.571429,0.354839,0.395349,0.350000
2,0.48,0.222222,0.379310,0.238095,0.153846,0.391304,0.271429,0.290323,0.255814,0.333333
3,0.20,0.055556,0.103448,0.571429,0.384615,0.217391,0.100000,0.161290,0.162791,0.116667
포착군집,2.00,1.000000,1.000000,3.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000


In [73]:
# 레이블 두개를 전부 사용한 경우의 평균 확률 -> 아예 label_data를 더해서 확률을 구하는 방법도 있음
eval_avg2 = (prob_df1.iloc[:-1,:] + prob_df2.iloc[:-1,:]) / 2
eval_avg2.loc["포착군집", :] = np.argmax(eval_avg2.values, axis=0)

eval_avg2

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.11500,0.204106,0.033114,0.200845,0.357453,0.197882,0.074624,0.218798,0.252114,0.187379
1,0.26375,0.504831,0.519157,0.128264,0.093655,0.241360,0.571898,0.350038,0.390856,0.359466
2,0.42750,0.187198,0.372195,0.175499,0.136245,0.375139,0.257425,0.282066,0.215028,0.326861
3,0.19375,0.103865,0.075534,0.495392,0.412647,0.185619,0.096053,0.149098,0.142001,0.126294
포착군집,2.00000,1.000000,1.000000,3.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000


In [74]:
eval_cluster_distance(eval_avg2, return_masked_data).sort_values("2012-01", axis=1, ascending=False)#.to_csv("1호_4_Y_거리.csv")

,당기순이익/총자본,영업이익/총자본,BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장/총자본,최대주주비율,거래량회전율,영업현금흐름/총자본,매출총이익/총자본,경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장,판매 및 관리비/총자본,타법인주식및출자증권취득결정자기자본대비_전체시장+차입금의존도,자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장/총자본,매출채권/총자본,최대주주비율_변동성
2012-01,0.583412,0.557525,0.255285,0.216177,0.214164,0.207579,0.207062,0.198054,0.162051,0.149889,0.149260,0.065749,0.033792
2013-01,0.162038,0.181502,0.607566,0.120748,0.258418,0.063533,0.160843,0.510312,0.159480,0.060897,0.613215,0.093999,0.007450
2014-01,0.375524,0.261452,0.167786,0.097847,0.104754,0.119281,0.511919,0.154641,0.502188,0.165611,0.180526,0.340444,0.018027
2015-01,0.301353,0.463281,0.180862,0.078719,0.096244,0.170815,0.641513,0.142774,0.449020,0.098030,0.181479,0.182952,0.012957
2016-01,0.258129,0.245921,0.301417,0.121958,0.129715,0.122233,0.262081,0.234113,0.364043,0.266351,0.354620,0.285960,0.053459
2017-01,0.302550,0.456136,0.169804,0.077214,0.091383,0.194782,0.649174,0.121822,0.487473,0.065062,0.167025,0.200063,0.017511
2018-01,0.229278,0.233031,0.544216,0.159623,0.200499,0.097585,0.204380,0.477990,0.243766,0.066405,0.447823,0.079475,0.015929
2019-01,0.220270,0.354782,0.194226,0.069261,0.050719,0.215537,0.616769,0.180252,0.490953,0.130505,0.194867,0.276805,0.005054
2020-01,0.144326,0.176723,0.587756,0.091956,0.085640,0.131322,0.194204,0.576550,0.173644,0.085139,0.572321,0.170160,0.010258
2021-01,0.215709,0.243924,0.222847,0.085298,0.016400,0.127525,0.427554,0.246106,0.555670,0.269173,0.226213,0.328736,0.034844


예측하는 거 만들기

사용할 기간을 주면 -> 이후 가능한 끝까지의 값을 예측하고, 결과를 리턴하기

In [75]:
# 이전의 Model로 이후의 데이터 클러스터링을 평가
def predict_cluster_result(model_save, raw_data, label_data,criterion_period:str, freq):
    if freq == 'Y':
        plus = 12
    elif freq == "Q":
        plus = 3
    date_idx = pd.date_range(criterion_period, "2020-01",freq=freq+'S')
    return_df = pd.DataFrame()
    for date in date_idx:
        period = pd.Period(str(date)[:7]) + plus
        date = str(date)[:7]
        eval = Optimizer.eval(model_save_together[date]["model"], scaled_together[str(period)], label_data_pp1[str(period)])
        df = pd.DataFrame(eval.values(), eval.keys(), columns=[str(period)+"예측"])
        return_df = pd.concat([return_df,df], axis=1)
    return return_df

predict_cluster_result(model_save_together, scaled_together,label_data_pp1, "2015-01", freq="Y")

,2016-01예측,2017-01예측,2018-01예측,2019-01예측,2020-01예측,2021-01예측
0,0.847458,0.089744,0.059211,0.589286,0.068182,0.067961
1,0.050847,0.820513,0.046053,0.029762,0.075758,0.087379
2,0.033898,0.051282,0.072368,0.041667,0.795455,0.116505
3,0.067797,0.038462,0.822368,0.339286,0.060606,0.728155


In [76]:
pd.DataFrame(model_save_together["2021-01"]["model"].predict(scaled_together["2022-07"])).value_counts()

KeyError: '2022-07'

### 코스닥 평가

In [ ]:
eval_cluster_size(model_save_kosdaq)

In [ ]:
eval_cluster_result(model_save_kosdaq, raw_data = scaled_kosdaq, label_data=label_data_pp1, freq='Y')

In [ ]:
eval_cluster_result(model_save_kosdaq, raw_data = scaled_kosdaq, label_data=label_data_pp2, freq='Y')

In [ ]:
eval_cluster_result(model_save_kosdaq, raw_data = scaled_kosdaq, label_data=label_data_pp3, freq='Y')

### 코스피결과

In [ ]:
eval_cluster_size(model_save_kospi)

In [ ]:
eval_cluster_result(model_save_kospi, raw_data = scaled_kospi, label_data=label_data_pp1, freq='Y')